In [1]:
import tester as t

import pandas as pd
import numpy as np

np.seterr(divide='ignore', invalid='ignore')

from surprise import SVD, SVDpp, KNNBasic, NMF

In [3]:
from movie_lens_data import get_movie_lens

ml_ratings_10m_df = get_movie_lens('10m')
ml_ratings_10m_df['timestamp'] = \
    pd.to_datetime(ml_ratings_10m_df['timestamp'], unit='s')
print(ml_ratings_10m_df.head(3))


   userId  movieId  rating           timestamp
0       1      122     5.0 1996-08-02 11:24:06
1       1      185     5.0 1996-08-02 10:58:45
2       1      231     5.0 1996-08-02 10:56:32


# SVD

In [4]:
def test_svd(df, splitter, filename, args_dict):
    tester = t.Tester(
        score_functions=t.all_score_functions, train_test_iter=splitter,
        df=df, df2ds=t.movielens_df2ds
    )

    tester.test([
        t.TestCase(alg=SVD(**args), args=args, filename=filename)
        for args in args_dict
    ])

def test_all_svd(df, filename, args):
    '''
    test_svd(
        df,
        t.CrossValidTrainTestProvider(),
        f'new_tests/cross_valid/{filename}.csv',
    )
    '''
    test_svd(
        df,
        t.NaiveTailTrainTestProvider(tail_len=int(0.1*len(df))),
        f'new_tests/tail_cut_off/{filename}.csv',
        args,
    )
    test_svd(
        df,
        t.ColdStartTrainTestProvider(users_num=5000, head_len=10),
        f'new_tests/cold_start/{filename}.csv',
        args,
    )

In [ ]:
svd_args_dict = [
    dict(
        n_factors=n_factors,
        n_epochs=n_epochs,
    )
    for n_factors in [15, 25, 50]
    for n_epochs in [5, 10, 25]
]

test_all_svd(ml_ratings_10m_df, 'SVD_10M', svd_args_dict)

Starting tests...
Number of tests: 9
Splitter: NaiveTailTrainTestProvider
Scoring functions: rmse, mse, mae, precision, rmse_4_5, mae_4_5, mae_4_5_unsym
Results will be saved to files.
.........
